# Lavorazioni interne

##### setup

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")


## 1) Scostamento costo totale Lavorazioni interne

In [ ]:
impiegoConsuntivo = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO' or budgetConsuntivo = 'Consuntivo'")
#join con i costi orari
impiegoConsuntivo  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoConsuntivo as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoConsuntivo = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')

In [ ]:
impiegoBudget = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget'")
#join con i costi orari
impiegoBudget  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoBudget as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
# righe a consuntivo sono di più che a budget e la somma dei 2 non corrisponde al numero di righe della tabella impiego risorse

# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoBudget = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')

In [ ]:
sumImpiegoConsuntivo = sqldf('SELECT sum(QuantitydiOutput*TempoRisorsa*CostoOrario) FROM impiegoConsuntivo')
sumImpiegoBudget = sqldf('SELECT sum(QuantitydiOutput*TempoRisorsa*CostoOrario) FROM impiegoBudget')
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget-sumImpiegoConsuntivo)

## 2) Scostamento di volume, impiego, prezzo

In [ ]:
scostamentoImpiegoVolume = sqldf(
''' SELECT sum(b.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario) as BudgetStandard, sum(c.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario) as ImpiegoStandard, sum(b.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario)-sum(c.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario) as scostamento
FROM impiegoConsuntivo as c join impiegoBudget as b 
on c.NrArticolo=b.NrArticolo and c.Risorsa=b.Risorsa and c.NrAreaProduzione=b.NrAreaProduzione
''')

scostamentoImpiegoVolume

,BudgetStandard,ImpiegoStandard,scostamento
0,6300260.412,5.016818e+06,1.283443e+06


In [ ]:
scostamentoImpiegoImpiego = sqldf(
''' SELECT  sum(c.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario) as ImpiegoStandard,sum(c.QuantitydiOutput*c.TempoRisorsa*b.CostoOrario) as ImpiegoEffettivo, sum(c.QuantitydiOutput*b.TempoRisorsa*b.CostoOrario)-sum(c.QuantitydiOutput*c.TempoRisorsa*b.CostoOrario) as scostamento
FROM impiegoConsuntivo as c join impiegoBudget as b 
on c.NrArticolo=b.NrArticolo and c.Risorsa=b.Risorsa and c.NrAreaProduzione=b.NrAreaProduzione
''')

scostamentoImpiegoImpiego

,ImpiegoStandard,ImpiegoEffettivo,scostamento
0,5.016818e+06,5.341534e+06,-324716.0676


In [ ]:
scostamentoImpiegoPrezzo = sqldf(
''' SELECT  sum(c.QuantitydiOutput*c.TempoRisorsa*b.CostoOrario) as ImpiegoEffettivo, sum(c.QuantitydiOutput*c.TempoRisorsa*c.CostoOrario) as BudgetEffettivo, sum(c.QuantitydiOutput*c.TempoRisorsa*b.CostoOrario)-sum(c.QuantitydiOutput*c.TempoRisorsa*c.CostoOrario) as scostamento
FROM impiegoConsuntivo as c join impiegoBudget as b 
on c.NrArticolo=b.NrArticolo and c.Risorsa=b.Risorsa and c.NrAreaProduzione=b.NrAreaProduzione
''')

scostamentoImpiegoPrezzo

,ImpiegoEffettivo,BudgetEffettivo,scostamento
0,5.341534e+06,5.341534e+06,0.0


In [ ]:
scostamentoImpiegoVolume.iloc[0,2]+scostamentoImpiegoImpiego.iloc[0,2]+scostamentoImpiegoPrezzo.iloc[0,2]

958726.5971999932